<img src="./img/sktime-logo-text-horizontal.jpg" alt="sktime logo" style="width: 100%; max-width: 600px;">

# Type 1 Diabetes - Meal Identification Change Point Benchmark

## The Purpose of the Challenge

Type 1 Diabetes (T1D) is a medical condition where the pancreas produces little to no insulin, the hormone that regulates blood glucose levels. 
Managing T1D for most is immensely challenging, the main goal is a delicate balance between:
1. Maximizing Time in Range (3.9mmol/L - 10.0 mmo/L blood glucose levels (BGL)),
2. Minimizing the short term risks caused by hypoglycemia (when BGL < 3.9mmol/L)
3. Minimizing the likelihood of [diabetes distress](https://www.diabetes.ca/about-diabetes-(3)/impact-stories/distressed-by-diabetes) or "diabetes burnout".
   
The best strategies to maximize TIR and minimize hypoglycemia risks usually involve meticulous tracking of carbohydrates consumed, and the amount of insulin taken before the meal. Insulin must be administered for nearly every meal that contains more than a 5-10g of carbs (exact amount varys by individual). Generally, it is not recommended that T1Ds discontinue consumption of carbohydrates, or risk [diabetic ketoacidosis](https://www.mayoclinic.org/diseases-conditions/diabetic-ketoacidosis/symptoms-causes/syc-20371551). The challenge with meticulously logging every meal is that it causes cognitive burden that increases the likelihood of diabetes distress. When observed outside of the context of the mental burden of meticulously tracking every meal, it leads to improved outcomes, it becomes much easier to estimate the correct timing and dosage quantity of insulin that will maximize your TIR if you have a detailed log because you can see how much insulin you took for similar previous meals.

Common questions evaluated on the [diabetes distress scale](https://diabetes.ca/DiabetesCanadaWebsite/media/Health-care-providers/2018%20Clinical%20Practice%20Guidelines/the-DDS-with-scoring-recommendations.pdf?ext=.pdf) where diabetics are asked to rate the following questions on a 6 point scale from *Not a Problem* to *A Very Serious Problem* (quoted from linked resource):

1. Feeling that diabetes is taking up too much of my mental and physical energy every day.
2. Not feeling confident in my day-to-day ability to manage diabetes.
3. Feeling that I am often failing with my diabetes routine. 
4. Feeling that diabetes controls my life. 
5. Feeling that I am not sticking closely enough to a good meal plan.
6. Feeling overwhelmed by the demands of living with diabetes. 
7. Not feeling motivated to keep up my diabetes self management. 

The focus of this benchmark is to provide a real-world usecase for creating tools that alleviate diabetes distress. 
A significant amount of research in the diabetes community now focuses on ways of reducing the severity and likelihood of diabetes distress. 
Our question is, what tools can we develop to make the logging of meals more streamlined and automated? 
Using a stream of data from [continuous glucose monitors (CGM)](https://www.niddk.nih.gov/health-information/diabetes/overview/managing-diabetes/continuous-glucose-monitoring), can we reliably and automatically identify meals in a blood glucose time series dataset? 
Possible usecases of this would be gentle notifications to the user at the end of a day identifying possible unlabelled meals, with a prepopulated change point log.

The result is that diabetics would have a more richly detailed log, with a reduced cognitive burden, but additionally researchers would have improved datasets that could lead to improvements in other downstream tasks like probabilistic hypoglycemic forecasting. 

## 


In [ ]:
from simglucose.simulation.env import T1DSimEnv
from simglucose.controller.basal_bolus_ctrller import BBController
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.simulation.scenario import CustomScenario
from simglucose.simulation.sim_engine import SimObj, sim, batch_sim
from datetime import timedelta
from datetime import datetime

# specify start_time as the beginning of today
now = datetime.now()
start_time = datetime.combine(now.date(), datetime.min.time())

# --------- Create Random Scenario --------------
# Specify results saving path
path = './results'

# Create a simulation environment
patient = T1DPatient.withName('adolescent#001')
sensor = CGMSensor.withName('Dexcom', seed=1)
pump = InsulinPump.withName('Insulet')
scenario = RandomScenario(start_time=start_time, seed=1)
env = T1DSimEnv(patient, sensor, pump, scenario)

# Create a controller
controller = BBController()

# Put them together to create a simulation object
s1 = SimObj(env, controller, timedelta(days=1), animate=False, path=path)
results1 = sim(s1)
print(results1)

# --------- Create Custom Scenario --------------
# Create a simulation environment
patient = T1DPatient.withName('adolescent#001')
sensor = CGMSensor.withName('Dexcom', seed=1)
pump = InsulinPump.withName('Insulet')
# custom scenario is a list of tuples (time, meal_size)
scen = [(7, 45), (12, 70), (16, 15), (18, 80), (23, 10)]
scenario = CustomScenario(start_time=start_time, scenario=scen)
env = T1DSimEnv(patient, sensor, pump, scenario)

# Create a controller
controller = BBController()

# Put them together to create a simulation object
s2 = SimObj(env, controller, timedelta(days=1), animate=False, path=path)
results2 = sim(s2)
print(results2)


# --------- batch simulation --------------
# Re-initialize simulation objects
s1.reset()
s2.reset()

# create a list of SimObj, and call batch_sim
s = [s1, s2]
results = batch_sim(s, parallel=True)
print(results)